In [1]:
#import
import pandas as pd
import numpy as np

#PATH dove andare a leggere e a scrivere
PATH_TO_WRITE_ETL = "./dataset_modificato/"
PATH_FROM_READ_ETL = "./dataset_originale/"
PATH_FROM_READ_ANALYSIS = "./dataset_modificato/" 

In [2]:
#INIZIO RAGGRUPPAMENTO DATI PER INIZIARE L'ANALISI SULLE NAZIONI

df_team_appearances = pd.read_csv(PATH_FROM_READ_ANALYSIS + "team_appearances.csv")
df_bookings = pd.read_csv(PATH_FROM_READ_ANALYSIS + "bookings.csv")
df_goals = pd.read_csv(PATH_FROM_READ_ANALYSIS + "goals.csv")
df_group_standings = pd.read_csv(PATH_FROM_READ_ANALYSIS + "group_standings.csv")
df_matches = pd.read_csv(PATH_FROM_READ_ANALYSIS + "matches.csv")
df_squads = pd.read_csv(PATH_FROM_READ_ANALYSIS + "squads.csv")
df_stadiums = pd.read_csv(PATH_FROM_READ_ANALYSIS + "stadiums.csv")
df_substitution = pd.read_csv(PATH_FROM_READ_ANALYSIS + "substitutions.csv")
df_tournaments = pd.read_csv(PATH_FROM_READ_ANALYSIS + "tournaments.csv")
df_tournament_stages = pd.read_csv(PATH_FROM_READ_ANALYSIS + "tournament_stages.csv")
df_tournament_standings = pd.read_csv(PATH_FROM_READ_ANALYSIS + "tournament_standings.csv")
df_menagers_appointments = pd.read_csv(PATH_FROM_READ_ANALYSIS + "manager_appointments.csv")
df_player = pd.read_csv(PATH_FROM_READ_ANALYSIS + "player_appearances.csv")

In [3]:
#CALCOLO CARTELLINI GIALLI, ROSSI ED ESPULSIONI PER OGNI PARTITA

match_data = df_matches[['match_name','home_team_id','home_team_name','home_team_code','away_team_id','away_team_name','away_team_code','score','home_team_score','away_team_score','home_team_score_margin','away_team_score_margin','home_team_score_penalties','away_team_score_penalties','home_team_win','away_team_win']]

booking_match_data = df_bookings.groupby(["match_name"])[['yellow_card','second_yellow_card','red_card']].agg('sum').reset_index().sort_values(by="match_name",ascending=True)

match_data = match_data.merge(booking_match_data, on="match_name", how="left")
match_data['yellow_card'] = match_data['yellow_card'].fillna(0).astype(int)
match_data['second_yellow_card'] = match_data['second_yellow_card'].fillna(0).astype(int)
match_data['red_card'] = match_data['red_card'].fillna(0).astype(int)

match_data['red_card'] = match_data['red_card'] + match_data['second_yellow_card']
match_data['yellow_card'] = match_data['yellow_card'] + match_data['second_yellow_card']

match_data = match_data.drop(columns=["second_yellow_card"])

sobstitutions_match_data = df_substitution.groupby(["match_name"])[['going_off']].agg('sum').reset_index().sort_values(by="match_name",ascending=True)
match_data = match_data.merge(sobstitutions_match_data, on="match_name", how="left")
match_data['going_off'] = match_data['going_off'].fillna(0).astype(int)
match_data = match_data.rename(columns={"going_off": "substitutions"})

match_data


,match_name,home_team_id,home_team_name,home_team_code,away_team_id,away_team_name,away_team_code,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,home_team_score_penalties,away_team_score_penalties,home_team_win,away_team_win,yellow_card,red_card,substitutions
0,France v Mexico,T-28,France,FRA,T-44,Mexico,MEX,4–1,4,1,3,-3,0,0,1,0,6,0,5
1,United States v Belgium,T-79,United States,USA,T-06,Belgium,BEL,3–0,3,0,3,-3,0,0,1,0,0,0,0
2,Yugoslavia v Brazil,T-83,Yugoslavia,YUG,T-09,Brazil,BRA,2–1,2,1,1,-1,0,0,1,0,0,0,0
3,Romania v Peru,T-58,Romania,ROU,T-54,Peru,PER,3–1,3,1,2,-2,0,0,1,0,0,1,0
4,Argentina v France,T-03,Argentina,ARG,T-28,France,FRA,1–0,1,0,1,-1,0,0,1,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,Russia v Croatia,T-59,Russia,RUS,T-17,Croatia,HRV,2–2,2,2,0,0,3,4,0,1,5,0,8
896,France v Belgium,T-28,France,FRA,T-06,Belgium,BEL,1–0,1,0,1,-1,0,0,1,0,5,0,5
897,Croatia v England,T-17,Croatia,HRV,T-27,England,ENG,2–1,2,1,1,-1,0,0,1,0,3,0,8
898,Belgium v England,T-06,Belgium,BEL,T-27,England,ENG,2–0,2,0,2,-2,0,0,1,0,3,0,6


In [4]:
#ORDINAMENTO SQUADRE CASA E FUORICASA

for squad in match_data.index:
        squad1 = match_data['home_team_id'][squad].split('-')
        squad2 = match_data['away_team_id'][squad].split('-')

        if squad1[1] > squad2[1]:
            match_name = match_data['match_name'][squad].split(' v ')
            match_data['match_name'][squad] = match_name[1] + ' v ' + match_name[0]

            score = match_data['score'][squad].split('–')
            match_data['score'][squad] = score[1] + '–' + score[0]

            code = match_data['home_team_code'][squad]
            match_data['home_team_code'][squad] = match_data['away_team_code'][squad]
            match_data['away_team_code'][squad] = code

            name = match_data['home_team_name'][squad]
            match_data['home_team_name'][squad] = match_data['away_team_name'][squad]
            match_data['away_team_name'][squad] = name

            id = match_data['home_team_id'][squad]
            match_data['home_team_id'][squad] = match_data['away_team_id'][squad]
            match_data['away_team_id'][squad] = id

            score_team = match_data['home_team_score'][squad]
            match_data['home_team_score'][squad] = match_data['away_team_score'][squad]
            match_data['away_team_score'][squad] = score_team

            margin = match_data['home_team_score_margin'][squad]
            match_data['home_team_score_margin'][squad] = match_data['away_team_score_margin'][squad]
            match_data['away_team_score_margin'][squad] = margin

            penalties = match_data['home_team_score_penalties'][squad]
            match_data['home_team_score_penalties'][squad] = match_data['away_team_score_penalties'][squad]
            match_data['away_team_score_penalties'][squad] = penalties

            win = match_data['home_team_win'][squad]
            match_data['home_team_win'][squad] = match_data['away_team_win'][squad]
            match_data['away_team_win'][squad] = win


match_data

C:\Users\mat98\AppData\Local\Temp\ipykernel_10708\1235065802.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_data['match_name'][squad] = match_name[1] + ' v ' + match_name[0]
C:\Users\mat98\AppData\Local\Temp\ipykernel_10708\1235065802.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_data['score'][squad] = score[1] + '–' + score[0]
C:\Users\mat98\AppData\Local\Temp\ipykernel_10708\1235065802.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

,match_name,home_team_id,home_team_name,home_team_code,away_team_id,away_team_name,away_team_code,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,home_team_score_penalties,away_team_score_penalties,home_team_win,away_team_win,yellow_card,red_card,substitutions
0,France v Mexico,T-28,France,FRA,T-44,Mexico,MEX,4–1,4,1,3,-3,0,0,1,0,6,0,5
1,Belgium v United States,T-06,Belgium,BEL,T-79,United States,USA,0–3,0,3,-3,3,0,0,0,1,0,0,0
2,Brazil v Yugoslavia,T-09,Brazil,BRA,T-83,Yugoslavia,YUG,1–2,1,2,-1,1,0,0,0,1,0,0,0
3,Peru v Romania,T-54,Peru,PER,T-58,Romania,ROU,1–3,1,3,-2,2,0,0,0,1,0,1,0
4,Argentina v France,T-03,Argentina,ARG,T-28,France,FRA,1–0,1,0,1,-1,0,0,1,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,Croatia v Russia,T-17,Croatia,HRV,T-59,Russia,RUS,2–2,2,2,0,0,4,3,1,0,5,0,8
896,Belgium v France,T-06,Belgium,BEL,T-28,France,FRA,0–1,0,1,-1,1,0,0,0,1,5,0,5
897,Croatia v England,T-17,Croatia,HRV,T-27,England,ENG,2–1,2,1,1,-1,0,0,1,0,3,0,8
898,Belgium v England,T-06,Belgium,BEL,T-27,England,ENG,2–0,2,0,2,-2,0,0,1,0,3,0,6


In [17]:
# AGGREGAZIONE

n_same_squad_encaunter = match_data.groupby(["match_name"]).size()\
             .reset_index(name="n_same_squad_match")

df_match_appearances = match_data.groupby(["match_name"])\
                                            [['home_team_score','away_team_score','yellow_card','red_card','substitutions']]\
                                            .agg('sum')\
                                            .reset_index()\
                                            .sort_values(by="match_name",ascending=True)

df_match_appearances = df_match_appearances.merge(n_same_squad_encaunter, on="match_name", how="left")

                    
df_match_appearances



,match_name,home_team_score,away_team_score,yellow_card,red_card,substitutions,n_same_squad_match
0,Algeria v Austria,0,2,1,0,4,1
1,Algeria v Belgium,1,2,2,0,6,1
2,Algeria v Brazil,0,1,0,0,4,1
3,Algeria v Chile,3,2,1,0,4,1
4,Algeria v England,0,0,2,0,6,1
...,...,...,...,...,...,...,...
610,United States v Yugoslavia,0,1,3,0,6,1
611,Uruguay v West Germany,1,6,10,4,12,3
612,Uruguay v Yugoslavia,7,4,0,2,0,2
613,West Germany v Yugoslavia,9,2,11,0,20,5


In [18]:
#SOMMA GOAL CASA E FUORICASA

df_match_appearances['home_team_score'] = df_match_appearances['home_team_score'] + df_match_appearances['away_team_score']
df_match_appearances = df_match_appearances.drop(columns=["away_team_score"])
df_match_appearances = df_match_appearances.rename(columns={"home_team_score": "total_teams_score"})

In [19]:
# TRASFORMO IN CSV

match_data.to_csv("./ETL_Mat/stats_match.csv",index=False)
df_match_appearances.to_csv("./ETL_Mat/stats_appearences.csv",index=False)